In [1]:
!pip install torch_geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 199.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 324.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [39]:
!pip install pandas matplotlib gdown


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 81 (delta 4), reused 4 (delta 4), pack-reused 70 (from 1)
Receiving objects: 100% (81/81), 105.83 MiB | 14.70 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (38/38), done.


In [3]:
%cd hackaton/

/workspace/hackaton


In [40]:
# !gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets
!gdown --folder https://drive.google.com/drive/folders/1vZVCPbux6brOnkfqil9CIj8a4iHlUwHV -O datasets

Retrieving folder contents
Retrieving folder 1oOxZgzm6GFEMqR9CE84syVkS3zDYm14G A
Processing file 1KK7CrI83nsEBG4ft_BgBeB3FPfrtqfs6 test.json.gz
Processing file 1HLDsT0NSttc9mI8obV4xsMdz0dXYsHg5 train.json.gz
Retrieving folder 1fZMQBg3Zkd9k8D3ExvMHLPDZ-A4Uldz3 B
Processing file 1nYi_Wj0c7FwMt1kNhenS5IOlfOAONUUO test.json.gz
Processing file 14bnCHnrPdLQ9fmLbnoi3GKFrwfHy2JPN train.json.gz
Retrieving folder 16I0ny5c6cuYncYTXNLro1i_bqUsJNeSc C
Processing file 1HftG39UqLt5Zu7sYHcDl8GN6PqZV7i_M test.json.gz
Processing file 1Q_QHBljWUS4ERBXbiZGQych-JKktRiMx train.json.gz
Retrieving folder 1ToGahOt-r0Llmo6Bwq6WHnzq31Y6ixwm D
Processing file 1yPgOhqJrmqP8eYfwCKgqdTz_blPZMzvj test.json.gz
Processing file 1oZy8-zEj8heFGWAJ2loD6adqpdARVMlA train.json.gz
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1KK7CrI83nsEBG4ft_BgBeB3FPfrtqfs6
From (redirected): https://drive.google.com/uc?

In [7]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import StepLR
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()


### Data loader

In [8]:
import gzip
import json
import torch
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree

class GraphDatasetWithFeatures(Dataset):
    def __init__(self, filename, transform=None, pre_transform=None):
        self.raw = filename
        self.num_graphs, self.graphs_dicts = self._count_graphs()
        super().__init__(None, transform, pre_transform)

    def len(self):
        return self.num_graphs

    def get(self, idx):
        return dictToGraphObjectWithFeatures(self.graphs_dicts[idx])

    def _count_graphs(self):
        with gzip.open(self.raw, "rt", encoding="utf-8") as f:
            graphs_dicts = json.load(f)
            return len(graphs_dicts), graphs_dicts


def dictToGraphObjectWithFeatures(graph_dict):
    edge_index = torch.tensor(graph_dict["edge_index"], dtype=torch.long)
    edge_attr = torch.tensor(graph_dict["edge_attr"], dtype=torch.float) if graph_dict.get("edge_attr") else None
    num_nodes = graph_dict["num_nodes"]
    y = torch.tensor(graph_dict["y"][0], dtype=torch.long) if graph_dict.get("y") is not None else None

    node_features = None
    if edge_attr is not None:
        node_feature_dim = edge_attr.shape[1]
        node_features = torch.zeros((num_nodes, node_feature_dim), dtype=torch.float)

        row, col = edge_index
        node_features.index_add_(0, row, edge_attr)
        node_features.index_add_(0, col, edge_attr)

        deg = degree(edge_index[0], num_nodes=num_nodes, dtype=torch.float)
        node_features = node_features / deg.unsqueeze(1)

    graph_features = torch.tensor(graph_dict["graph_features"], dtype=torch.float) if graph_dict.get("graph_features") else None

    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, num_nodes=num_nodes, y=y, graph_x=graph_features)

### Model

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv, GCNConv, global_add_pool

class GNN_node_Virtualnode_With_Features(torch.nn.Module):
    """
    Output:
        node representations
    """
    def __init__(self, num_layer, emb_dim, drop_ratio=0.5, JK="last", residual=False, gnn_type='gin', input_dim=None):
        '''
            emb_dim (int): node embedding dimensionality (output dimension of the first layer, and hidden dim for subsequent layers)
            input_dim (int): the actual dimension of the input node features (e.g., 7 for OGB-PPA derived features)
        '''
        super(GNN_node_Virtualnode_With_Features, self).__init__()
        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        self.residual = residual
        self.emb_dim = emb_dim # This is the hidden dimension of the GNN layers

        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        if input_dim is None:
            raise ValueError("input_dim must be provided for GNN_node_Virtualnode (e.g., 7 for OGB-PPA).")
        self.node_encoder = torch.nn.Linear(input_dim, emb_dim)

        ### set the initial virtual node embedding to 0.
        self.virtualnode_embedding = torch.nn.Embedding(1, emb_dim)
        torch.nn.init.constant_(self.virtualnode_embedding.weight.data, 0)

        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()

        self.mlp_virtualnode_list = torch.nn.ModuleList()

        for layer in range(num_layer):
            # PyG's GINConv and GCNConv expect `in_channels` and `out_channels`
            # For GINConv, it takes an MLP, so we pass `emb_dim` as the input/output dim of the MLP.
            # For GCNConv, it takes `in_channels` and `out_channels`.
            if gnn_type == 'gin':
                # The MLP passed to GINConv should map from emb_dim to emb_dim
                self.convs.append(GINConv(torch.nn.Sequential(
                    torch.nn.Linear(emb_dim, 2 * emb_dim),
                    torch.nn.BatchNorm1d(2 * emb_dim),
                    torch.nn.ReLU(),
                    torch.nn.Linear(2 * emb_dim, emb_dim)
                )))
            elif gnn_type == 'gcn':
                self.convs.append(GCNConv(emb_dim, emb_dim))
            else:
                raise ValueError('Undefined GNN type called {}'.format(gnn_type))

            self.batch_norms.append(torch.nn.BatchNorm1d(emb_dim))

        for layer in range(num_layer - 1):
            self.mlp_virtualnode_list.append(torch.nn.Sequential(
                torch.nn.Linear(emb_dim, 2*emb_dim),
                torch.nn.BatchNorm1d(2*emb_dim),
                torch.nn.ReLU(),
                torch.nn.Linear(2*emb_dim, emb_dim),
                torch.nn.BatchNorm1d(emb_dim),
                torch.nn.ReLU() # ReLU here is consistent with the original
            ))

    def forward(self, batched_data):
        x, edge_index, edge_attr, batch = batched_data.x, batched_data.edge_index, batched_data.edge_attr, batched_data.batch

        if x is None:
            raise ValueError("batched_data.x (node features) is None. Ensure your dataset correctly derives them.")
        h = self.node_encoder(x)
        h_list = [h]

        virtualnode_embedding = self.virtualnode_embedding(torch.zeros(batch[-1].item() + 1, dtype=torch.long, device=edge_index.device))

        for layer in range(self.num_layer):
            h_list[layer] = h_list[layer] + virtualnode_embedding[batch]

            if self.convs[layer].__class__.__name__ == 'GINConv':
                h = self.convs[layer](h_list[layer], edge_index) 
            elif self.convs[layer].__class__.__name__ == 'GCNConv':
                h = self.convs[layer](h_list[layer], edge_index, edge_weight=edge_attr if edge_attr is not None else None) # GCNConv can take edge_weight
            else:
                h = self.convs[layer](h_list[layer], edge_index, edge_attr) 

            h = self.batch_norms[layer](h)

            if layer == self.num_layer - 1:
                h = F.dropout(h, self.drop_ratio, training=self.training)
            else:
                h = F.dropout(F.relu(h), self.drop_ratio, training=self.training)

            if self.residual:
                h = h + h_list[layer] # Residual connection

            h_list.append(h)

            ### update the virtual nodes
            if layer < self.num_layer - 1:
                virtualnode_embedding_temp = global_add_pool(h_list[layer], batch) + virtualnode_embedding
                ### transform virtual nodes using MLP

                if self.residual:
                    virtualnode_embedding = virtualnode_embedding + F.dropout(self.mlp_virtualnode_list[layer](virtualnode_embedding_temp), self.drop_ratio, training=self.training)
                else:
                    virtualnode_embedding = F.dropout(self.mlp_virtualnode_list[layer](virtualnode_embedding_temp), self.drop_ratio, training=self.training)

        ### Different implementations of Jk-concat
        if self.JK == "last":
            node_representation = h_list[-1]
        elif self.JK == "sum":
            node_representation = 0
            # Iterate over all h_list (including initial embedding)
            for h_rep in h_list:
                node_representation += h_rep
        else:
            raise ValueError("Undefined JK type: {}".format(self.JK))

        return node_representation

In [10]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool, GlobalAttention, Set2Set
import torch.nn.functional as F
from torch_geometric.nn.inits import uniform

from src.conv import GNN_node, GNN_node_Virtualnode

class GNN_With_Features(torch.nn.Module):
    def __init__(self, num_class, num_layer=5, emb_dim=None, # <--- emb_dim can be dynamically set
                 gnn_type='gin', virtual_node=True, residual=False,
                 drop_ratio=0.5, JK="last", graph_pooling="mean",
                 num_graph_features=0):
        super(GNN_With_Features, self).__init__()
        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        self.num_class = num_class
        self.graph_pooling = graph_pooling
        self.num_graph_features = num_graph_features

        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        # If emb_dim is not provided, we will determine it from the first batch
        # This requires `self.gnn_node` to handle None initially or be initialized later
        # A safer approach is to pass the expected input dim to GNN_node
        # For OGB-PPA, the input node feature dimension is 7 (from edge_attr)
        self.input_node_feat_dim = 7 # <--- Explicitly set for OGB-PPA derived features
        self.emb_dim = emb_dim if emb_dim is not None else self.input_node_feat_dim # Use derived dim if not specified

        # GNN_node now needs to know the input dimension (self.input_node_feat_dim)
        if virtual_node:
            self.gnn_node = GNN_node_Virtualnode_With_Features(
                num_layer,
                self.emb_dim,
                JK=JK,
                drop_ratio=drop_ratio,
                residual=residual,
                gnn_type=gnn_type,
                input_dim=self.input_node_feat_dim) # <--- Pass input_dim
        else:
            self.gnn_node = GNN_node(num_layer, self.emb_dim, JK=JK, drop_ratio=drop_ratio, residual=residual, gnn_type=gnn_type,
                                    input_dim=self.input_node_feat_dim) # <--- Pass input_dim

        ### Pooling function
        if self.graph_pooling == "sum":
            self.pool = global_add_pool
        elif self.graph_pooling == "mean":
            self.pool = global_mean_pool
        elif self.graph_pooling == "max":
            self.pool = global_max_pool
        elif self.graph_pooling == "attention":
            # Gate network input dimension needs to match emb_dim
            self.pool = GlobalAttention(gate_nn = torch.nn.Sequential(torch.nn.Linear(self.emb_dim, 2*self.emb_dim), torch.nn.BatchNorm1d(2*self.emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*self.emb_dim, 1)))
        elif self.graph_pooling == "set2set":
            self.pool = Set2Set(self.emb_dim, processing_steps = 2)
        else:
            raise ValueError("Invalid graph pooling type.")

        # Final linear layer
        if graph_pooling == "set2set":
            pooled_dim = 2 * self.emb_dim
        else:
            pooled_dim = self.emb_dim

        self.graph_pred_linear = torch.nn.Linear(pooled_dim + self.num_graph_features, self.num_class)


    def forward(self, batched_data):
        # The GNN_node module expects batched_data.x for node features
        # Your dictToGraphObject now correctly sets batched_data.x
        h_node = self.gnn_node(batched_data)
        h_graph = self.pool(h_node, batched_data.batch)

        if self.num_graph_features > 0:
            if not hasattr(batched_data, 'graph_x') or batched_data.graph_x is None:
                raise ValueError("num_graph_features is set > 0, but 'graph_x' attribute not found in batched_data.")
            if batched_data.graph_x.shape[0] != h_graph.shape[0]:
                raise ValueError(f"Batch size of graph_x ({batched_data.graph_x.shape[0]}) does not match pooled graph embeddings ({h_graph.shape[0]}).")

            h_graph = torch.cat([h_graph, batched_data.graph_x], dim=-1)

        return self.graph_pred_linear(h_graph)

### Loss

In [11]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class GeneralizedCrossEntropyLoss(nn.Module):
    """
    Generalized Cross Entropy Loss for Training Deep Neural Networks with Noisy Labels
    2018 - Zhang et al. - https://arxiv.org/pdf/1805.07836.pdf
    
    Args:
        q (float): Robustness parameter. q=0 gives standard cross-entropy, q=1 gives MAE.
                  Recommended range: [0.1, 0.7] for noise robustness.
        reduction (str): Specifies the reduction to apply to the output.
        eps (float): Small value to prevent numerical instability.
    """
    
    def __init__(self, q=0.3, reduction='mean', eps=1e-7):
        super(GeneralizedCrossEntropyLoss, self).__init__()
        self.q = q
        self.reduction = reduction
        self.eps = eps
    
    def forward(self, logits, targets):
        """
        Args:
            logits: Tensor of shape (N, C) containing raw logits
            targets: Tensor of shape (N,) containing class indices
        """
        # Convert logits to probabilities
        y_pred = F.softmax(logits, dim=-1)
        
        # Convert targets to one-hot encoding
        y_true = F.one_hot(targets, num_classes=logits.size(-1)).float()
        
        # Compute GCE loss with clamping for numerical stability
        # Clamp predictions to avoid numerical issues
        y_pred = torch.clamp(y_pred, min=self.eps, max=1.0 - self.eps)
        
        # Compute the sum of y_true * y_pred along class dimension
        pred_sum = torch.sum(y_true * y_pred, dim=-1)
        
        # Clamp again before taking the power to ensure stability
        pred_sum = torch.clamp(pred_sum, min=self.eps)
        
        # GCE loss: (1 - pred_sum^q) / q
        if self.q == 0:
            # Special case: standard cross-entropy (limit as q->0)
            loss = -torch.log(pred_sum)
        else:
            intermed = torch.pow(pred_sum, self.q)
            loss = (1 - intermed) / self.q
        
        if self.reduction == 'mean':
            return torch.mean(loss)
        elif self.reduction == 'sum':
            return torch.sum(loss)
        else:  # 'none'
            return loss

In [70]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch, scaler=None):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [13]:
def evaluate(data_loader, model, device, criterion=None, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                if criterion is not None:
                    total_loss += criterion(output, data.y).item()

            else:
                predictions.extend(pred.cpu().numpy())
    
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader), accuracy
        
    return predictions

In [96]:
def save_predictions(predictions, test_path):
    # script_dir = os.path.dirname(os.path.abspath(__file__))
    script_dir = os.getcwd()
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [15]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [55]:
# script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = os.getcwd()
# device = torch.device("cuda:1" if torch.cuda.is_available() else torch.device("cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = 5

In [166]:
class Args():
    def __init__(
            self,
            dataset_name,
            noise_ratio=0.2,
            num_layer=5,
            emb_dim=256,
            drop_ratio=0.2,
            batch_size=32,
    ):
        self.train_path = f"datasets/{dataset_name}/train.json.gz"
        self.test_path = f"datasets/{dataset_name}/test.json.gz"

        self.noise_ratio = noise_ratio

        self.num_layer = num_layer
        self.emb_dim = emb_dim
        self.drop_ratio = drop_ratio
        self.batch_size = batch_size
        

In [167]:
args = Args("D")

In [168]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))

In [152]:
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())  # Console output as well

In [161]:
# Define checkpoint path relative to the script's directory
checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)

In [37]:
# Load pre-trained model for inference
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [169]:
model = GNN_With_Features(
    gnn_type='gin',
    num_class=6,
    num_layer=args.num_layer,
    emb_dim=args.emb_dim,
    drop_ratio=args.drop_ratio,
    virtual_node=True,
    residual=True,
).to(device)

In [170]:
criterion = GeneralizedCrossEntropyLoss(args.noise_ratio)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

In [41]:
full_dataset = GraphDatasetWithFeatures(args.train_path)

In [171]:
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size

generator = torch.Generator().manual_seed(12)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=8, pin_memory=True)

In [172]:
num_epochs = 100
best_accuracy = 0.0

train_losses = []
train_accuracies = []

val_losses = []
val_accuracies = []

if num_checkpoints > 1:
    checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
else:
    checkpoint_intervals = [num_epochs]

for epoch in range(num_epochs):
    train_loss, train_acc = train(
        train_loader,
        model, 
        optimizer, 
        criterion, 
        device,
        save_checkpoints=(epoch + 1 in checkpoint_intervals),
        checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
        current_epoch=epoch,
    )
    
    val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True, criterion=criterion)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc {val_acc:.4f}")

    scheduler.step()
    
    # Save logs for training progress
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc {val_acc:.4f}")

    # Save best model
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Best model updated and saved at {checkpoint_path}")

# Plot training progress in current directory
plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Iterating eval graphs: 100%|██████████| 65/65 [00:05<00:00, 11.83batch/s]
Epoch 1/100, Loss: 2.3773, Val loss: 1.5184, Train Acc: 0.3479, Val Acc 0.3813
Epoch 1/100, Loss: 2.3773, Val loss: 1.5184, Train Acc: 0.3479, Val Acc 0.3813
Epoch 1/100, Loss: 2.3773, Val loss: 1.5184, Train Acc: 0.3479, Val Acc 0.3813
Epoch 1/100, Loss: 2.3773, Val loss: 1.5184, Train Acc: 0.3479, Val Acc 0.3813
Epoch 1/100, Loss: 2.3773, Val loss: 1.5184, Train Acc: 0.3479, Val Acc 0.3813
Epoch 1/100, Loss: 2.3773, Val loss: 1.5184, Train Acc: 0.3479, Val Acc 0.3813


Epoch 1/100, Loss: 2.3773, Val loss: 1.5184, Train Acc: 0.3479, Val Acc 0.3813
Best model updated and saved at /workspace/hackaton/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:05<00:00, 11.46batch/s]
Epoch 2/100, Loss: 1.3245, Val loss: 1.1622, Train Acc: 0.4190, Val Acc 0.4329
Epoch 2/100, Loss: 1.3245, Val loss: 1.1622, Train Acc: 0.4190, Val Acc 0.4329
Epoch 2/100, Loss: 1.3245, Val loss: 1.1622, Train Acc: 0.4190, Val Acc 0.4329
Epoch 2/100, Loss: 1.3245, Val loss: 1.1622, Train Acc: 0.4190, Val Acc 0.4329
Epoch 2/100, Loss: 1.3245, Val loss: 1.1622, Train Acc: 0.4190, Val Acc 0.4329
Epoch 2/100, Loss: 1.3245, Val loss: 1.1622, Train Acc: 0.4190, Val Acc 0.4329


Epoch 2/100, Loss: 1.3245, Val loss: 1.1622, Train Acc: 0.4190, Val Acc 0.4329
Best model updated and saved at /workspace/hackaton/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:05<00:00, 11.72batch/s]
Epoch 3/100, Loss: 1.1493, Val loss: 1.2755, Train Acc: 0.4688, Val Acc 0.4523
Epoch 3/100, Loss: 1.1493, Val loss: 1.2755, Train Acc: 0.4688, Val Acc 0.4523
Epoch 3/100, Loss: 1.1493, Val loss: 1.2755, Train Acc: 0.4688, Val Acc 0.4523
Epoch 3/100, Loss: 1.1493, Val loss: 1.2755, Train Acc: 0.4688, Val Acc 0.4523
Epoch 3/100, Loss: 1.1493, Val loss: 1.2755, Train Acc: 0.4688, Val Acc 0.4523
Epoch 3/100, Loss: 1.1493, Val loss: 1.2755, Train Acc: 0.4688, Val Acc 0.4523


...

Checkpoint saved at /workspace/hackaton/checkpoints/D/model_D_epoch_80.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:05<00:00, 12.11batch/s]
Epoch 80/100, Loss: 0.3165, Val loss: 0.6603, Train Acc: 0.8354, Val Acc 0.6751
Epoch 80/100, Loss: 0.3165, Val loss: 0.6603, Train Acc: 0.8354, Val Acc 0.6751
Epoch 80/100, Loss: 0.3165, Val loss: 0.6603, Train Acc: 0.8354, Val Acc 0.6751
Epoch 80/100, Loss: 0.3165, Val loss: 0.6603, Train Acc: 0.8354, Val Acc 0.6751
Epoch 80/100, Loss: 0.3165, Val loss: 0.6603, Train Acc: 0.8354, Val Acc 0.6751
Epoch 80/100, Loss: 0.3165, Val loss: 0.6603, Train Acc: 0.8354, Val Acc 0.6751


Epoch 80/100, Loss: 0.3165, Val loss: 0.6603, Train Acc: 0.8354, Val Acc 0.6751


Iterating training graphs:  23%|██▎       | 60/257 [00:07<00:23,  8.44batch/s]


KeyboardInterrupt: 

In [102]:
full_loader = DataLoader(full_dataset, batch_size=args.batch_size, shuffle=True, num_workers=8, pin_memory=True)

In [105]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

In [106]:
model.load_state_dict(torch.load(checkpoint_path))

num_epochs = 10

num_checkpoints = 10

if num_checkpoints > 1:
    checkpoint_intervals = [50 + int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
else:
    checkpoint_intervals = [num_epochs]

for epoch in range(num_epochs):
    train_loss, train_acc = train(
        full_loader,
        model, 
        optimizer, 
        criterion, 
        device,
        save_checkpoints=(50 + epoch + 1 in checkpoint_intervals),
        checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
        current_epoch=epoch,
    )
    
    print(f"Epoch {epoch + 51}/{num_epochs + 50}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    # Save logs for training progress
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    logging.info(f"Epoch {epoch + 51}/{num_epochs + 50}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")


torch.save(model.state_dict(), checkpoint_path)
print(f"Best model updated and saved at {checkpoint_path}")

# Plot training progress in current directory
plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Iterating training graphs: 100%|██████████| 322/322 [00:18<00:00, 17.79batch/s]
Epoch 51/60, Loss: 0.4312, Train Acc: 0.7606


Checkpoint saved at /workspace/hackaton/checkpoints/D/model_D_epoch_1.pth
Epoch 51/60, Loss: 0.4312, Train Acc: 0.7606


Iterating training graphs: 100%|██████████| 322/322 [00:15<00:00, 20.63batch/s]
Epoch 52/60, Loss: 0.4232, Train Acc: 0.7700


Checkpoint saved at /workspace/hackaton/checkpoints/D/model_D_epoch_2.pth
Epoch 52/60, Loss: 0.4232, Train Acc: 0.7700


Iterating training graphs: 100%|██████████| 322/322 [00:17<00:00, 18.37batch/s]
Epoch 53/60, Loss: 0.4210, Train Acc: 0.7707


...

Checkpoint saved at /workspace/hackaton/checkpoints/D/model_D_epoch_8.pth
Epoch 58/60, Loss: 0.3912, Train Acc: 0.7822


Iterating training graphs: 100%|██████████| 322/322 [00:18<00:00, 17.79batch/s]
Epoch 59/60, Loss: 0.3915, Train Acc: 0.7874


Checkpoint saved at /workspace/hackaton/checkpoints/D/model_D_epoch_9.pth
Epoch 59/60, Loss: 0.3915, Train Acc: 0.7874


Iterating training graphs: 100%|██████████| 322/322 [00:16<00:00, 19.49batch/s]
Epoch 60/60, Loss: 0.3868, Train Acc: 0.7913


Checkpoint saved at /workspace/hackaton/checkpoints/D/model_D_epoch_10.pth
Epoch 60/60, Loss: 0.3868, Train Acc: 0.7913
Best model updated and saved at /workspace/hackaton/checkpoints/model_D_best.pth


In [88]:
test_dataset = GraphDatasetWithFeatures(args.test_path)

In [89]:
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=8)

In [91]:
checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")

In [173]:
model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

In [175]:
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)

Iterating eval graphs: 100%|██████████| 71/71 [00:05<00:00, 12.79batch/s]


In [176]:
save_predictions(predictions, args.test_path)

Predictions saved to /workspace/hackaton/submission/testset_D.csv


In [116]:
# best Epoch 47/50, Loss: 0.4592, Val loss: 0.5975, Train Acc: 0.7461, Val Acc 0.6941
# needs retrain

# num_layer=5,
# emb_dim=300,
# noize 0.3
# without training on the full dataset

# optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-4)
# scheduler = StepLR(optimizer, step_size=10, gamma=0.8)

In [98]:
!pwd

/workspace/hackaton


In [100]:
import tarfile
import os

In [177]:
def gzip_folder(folder_path, output_file):
    """
    Compresses an entire folder into a single .tar.gz file.
    
    Args:
        folder_path (str): Path to the folder to compress.
        output_file (str): Path to the output .tar.gz file.
    """
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
    print(f"Folder '{folder_path}' has been compressed into '{output_file}'")
    

# Example usage
folder_path = "./submission"            # Path to the folder you want to compress
output_file = "./submission.gz"         # Output .gz file name
gzip_folder(folder_path, output_file)

Folder './submission' has been compressed into './submission.gz'
